# Challenge TextRecognition Apprentissage Profond
Auteur : JMLD - AD - CN
Date : 30/01/2017

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random

import zipfile
import collections

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import random

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

In [40]:
datapath = "../IntroductionDL/"
trainf = "calldesk-intents.txt"
dataf = datapath + trainf
train = pd.read_csv(dataf,names=['Cible','Mot1','Mot2','Mot3','Mot4','Mot5','Mot6','Mot7','Mot8','Mot9','Mot10'],dtype=str,delim_whitespace=True)
train.dtypes

Cible    object
Mot1     object
Mot2     object
Mot3     object
Mot4     object
Mot5     object
Mot6     object
Mot7     object
Mot8     object
Mot9     object
Mot10    object
dtype: object

In [3]:
train.head()

,Cible,Mot1,Mot2,Mot3,Mot4,Mot5,Mot6,Mot7,Mot8,Mot9,Mot10
0,Confirm,OK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Confirm,absolument,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Confirm,ah,oui,ça,c,est,bien,ça,comme,ça,NaN
3,Confirm,aller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Confirm,allez,c,est,bon,pour,$TIME$,NaN,NaN,NaN,NaN


In [4]:
le.fit(train['Cible'])
list(le.classes_)
train['Cible'] = le.transform(train['Cible'])

#pd.get_dummies
train_Mot1 = pd.get_dummies(train['Mot1'])
train_Mot2 = pd.get_dummies(train['Mot2'])
train_Mot3 = pd.get_dummies(train['Mot3'])
train_Mot4 = pd.get_dummies(train['Mot4'])
train_Mot5 = pd.get_dummies(train['Mot5'])
train_Mot6 = pd.get_dummies(train['Mot6'])
train_Mot7 = pd.get_dummies(train['Mot7'])
train_Mot8 = pd.get_dummies(train['Mot8'])
train_Mot9 = pd.get_dummies(train['Mot9'])
train_Mot10 = pd.get_dummies(train['Mot10'])
train_new = pd.concat([train,train_Mot1,train_Mot2,train_Mot3,train_Mot4,train_Mot5,train_Mot6,train_Mot7,train_Mot8,train_Mot9,train_Mot10],axis=1)

#Suppression des doublons

del train_new['Mot1']
del train_new['Mot2']
del train_new['Mot3']
del train_new['Mot4']
del train_new['Mot5']
del train_new['Mot6']
del train_new['Mot7']
del train_new['Mot8']
del train_new['Mot9']
del train_new['Mot10']


#del train_new['Finess']

In [5]:
train_new.head()

,Cible,$TIME$,OK,a-t-il,absolument,accessible,ah,ajouter,aller,allez,...,prochain,prochains,que,rendez-vous,revoir,s,te,une,voir,vous
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_new.iloc[2][train_new.iloc[2]>0]

ah       1
oui      1
ça       1
c        1
est      1
bien     1
ça       1
comme    1
ça       1
Name: 2, dtype: int64

In [7]:
import random
def kfolds(k,N,seed=None):
    random.seed(seed)
    out = [ list() for _ in range(k) ]
    for n in range(N): out[random.randrange(k)].append(n)
    return(out)

K = kfolds(seed=42,k=5,N=len(train_new))
fold1 = sum(K[1:5],[]) # K[1] + K[2] + K[3] + K[4]
#fold1 = K[1]+K[2]+K[3]+K[4]
xtrain = train_new.iloc[fold1][train_new.keys()[1:]]
ytrain = train_new.iloc[fold1][train_new.keys()[0]]

xtest = train_new.iloc[K[0]][train_new.keys()[1:]]
ytest = train_new.iloc[K[0]][train_new.keys()[0]]

In [8]:
from sklearn.ensemble import RandomForestClassifier
#
clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)

In [9]:
fitclf = clf.fit(xtrain,ytrain)

In [10]:
fitclf.score(xtest,ytest)

0.87341772151898733

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()

In [12]:
fitgbc = gbc.fit(xtrain,ytrain)

In [13]:
fitgbc.score(xtest,ytest)

0.83544303797468356

# Use of nltk to improve the data cleaning

In [28]:
import nltk
from nltk import word_tokenize
#nltk.download()

In [29]:
import snowballstemmer

stemmer = snowballstemmer.stemmer('french');
print(stemmer.stemWords("Nous sommes le monde".split()));

['Nous', 'somm', 'le', 'mond']


In [67]:
train =  pd.DataFrame({('a'): {('A'): 'vache', ('B'): 'vache'},
   ....:               ('b'): {('A'): 'cochon', ('C'): 'uluberlu'},
   ....:               ('c'): {('A'): 'cochonne', ('D'): 'bouzouk'},
   ....:               ('a'): {('B'): 'vive le vent dhiver', ('C'): 'pirate'},
   ....:               ('b'): {('B'): 'loup', ('C'): 'aretes'}})
   ....: 

In [68]:
train

,a,b,c
A,NaN,NaN,cochonne
B,vive le vent dhiver,loup,NaN
C,pirate,aretes,NaN
D,NaN,NaN,bouzouk


In [69]:
train.dtypes

a    object
b    object
c    object
dtype: object

In [74]:
def stemming(x) :
    return stemmer.stemWords(x.split())

In [75]:
stemming(train['a']['C'])

['pirat']

In [78]:
train['a'] = train['a'].apply(stemming(train['a']['A']))

AttributeError: 'float' object has no attribute 'split'

In [23]:
tokens = word_tokenize('vache aze sdfsdf zerzer poulet')

In [24]:
tokens

['vache', 'aze', 'sdfsdf', 'zerzer', 'poulet']

In [25]:
lancaster = nltk.LancasterStemmer()

In [26]:
[lancaster.stem(t) for t in tokens]

['vach', 'az', 'sdfsdf', 'zerz', 'poulet']

In [31]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

In [25]:
train.head()

,Cible,Mot1,Mot2,Mot3,Mot4,Mot5,Mot6,Mot7,Mot8,Mot9,Mot10
0,0,OK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,absolument,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,ah,oui,ça,c,est,bien,ça,comme,ça,NaN
3,0,aller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,allez,c,est,bon,pour,$TIME$,NaN,NaN,NaN,NaN


In [61]:
l = len(train['Mot1'])
l

800

In [75]:
#tokenstext = word_tokenize(train['Mot1'][1])
tokenstext

[['OK'],
 ['absolument'],
 ['ah'],
 ['aller'],
 ['allez'],
 ['alors'],
 ['bah'],
 ['ben'],
 ['bon'],
 ['bon'],
 ['c'],
 ['c'],
 ['entendu'],
 ['il'],
 ['iphone'],
 ['j'],
 ['je'],
 ['je'],
 ['moi'],
 ['moi'],
 ['non'],
 ['ok'],
 ['on'],
 ['ouais'],
 ['ouais'],
 ['ouais'],
 ['ouais'],
 ['ouais'],
 ['ouais'],
 ['ouais'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['oui'],
 ['parfait'],
 ['si'],
 ['très'],
 ['voilà'],
 ['voilà'],
 ['voilà'],
 ['à'],
 ['ça'],
 ['ça'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['$', 'TIME', '$'],
 ['ah'],
 ['ah'],
 ['ah'],
 ['ah'],
 ['ah'],
 ['ah'],
 ['ah'],
 ['ah'],
 ['alors'],
 ['avez-vous'],
 ['bon'],
 ['c'],
 ['c'],
 ['c'],
 ['c'],
 ['c'],
 ['

In [74]:
tokenstext = []
for i in range (0,l):
    print(i)
    tokenstext.append(word_tokenize(train['Mot1'][i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

TypeError: expected string or bytes-like object

In [58]:
[lancaster.stem(t) for t in tokenstext]

['absolu']

In [48]:
train['Mot1'] = train["Mot1"].apply(lambda x: [stemmer.stem(y) for y in x])

TypeError: 'float' object is not iterable